**Challenge Project v.2**

---

This version allows for much more freedom, but more grading .. with only 22-23 students, I'm fine with that
the success of this version will be determined by how active, excited, comfortable the students are (will help prepare for week 4, too!!) (Thanks Mr. Rod for the inspiration)

* not only helps build and enforce their knowledge of the curriculum, but also public speaking skills, appropriate statistical proceduring and considerations, etc. 

* Idea founded in the expression: "we remember 10% of what we read, 20% of what we hear, 30% of what we see, 50% of what we see and hear, 70% of what we discuss with others, 80% of what we personally experience
95% or what we teach others". So why not just teach?

* group or individual... or assigned group?

* students can only use algorithms learned so far (classification)

* may need to increase allotted time for group presentations (currently is 1 hour, 1 hour extra is probably enough) 

* take proposals from students first? or just assign research questions to each of the datasets?

The students will be tasked with taking a dataset and performing a full-on analysis procedure of the data, from the data cleaning to model evaluation. By giving the students an extensive, on-hands project, the aim is to reinforce their knowledge of the course curriculum, test their ability to accurately perform a typical analysis procedure, improve their communication and public speaking skills, and gain valuable insights into what is expected of them in this field.

Rough guideline of project:

>1.   Acquire/choose data
2.   Clean data as necessary
3.   further data visualization, analysis
4.   Select best model
>>4. Compare with other models
5.   Evaluate model
>>5. Sensitivity, specificity?
6.   Conclusion (any insights you can make?)
7.   Presentation?? (8-12 minutes)



Hypothesis Testing

H.P.F.P.D.C.

>**H**   Hypothesis. State null, alternate hypotheses. In case of classification model selection, the alternate would be the performance of the model you selected, the null would be the other models. Could probably just run a chi-squared GOF test if metric is a proportion.

>>in case of a specific model, the alternate would be the model you are using, and the null would be 0 or the proportion if one were to guess "no" for all or "yes" (depending on which value is higher) (in the case of AUROC, null would be 0.5)

>**P** Preconditions. Data is unbiased (therefore the true center isn't), large sample (n>30), records are independent (therefore variance in data is not skewed)

>**F** Formula. Depends on how you are comparing values (Chi-squared? 1-Prop z-test?)

>**P** Probability/P-value. The probability of receiving these results assuming the null hypothesis was true. If the p-value was low, that means that if the null hypothesis was true, then a miracle just happened, so it's better to say instead that the null hypothesis was "false."

>**D** Decision. Do you reject/fail to reject null?

>**C** Conclusion. Is there enough evidence to suggest that the models are similar in performance/better/worse?


Why is hypothesis testing important? Well, let's do a quick demonstration: Show of hands, who thinks 50/100 is different enough from 50/100 so that it matters? 51? 52? 53?...60? As you can see, not everyone agreed on when two numbers were considered "different enough." Hypothesis testing, statistics in general, really, objectively quantifies this discrepancy and therefore eliminates it. It allows for coherent structuring of a question, procedure, and answer universally understandable. That said, there is a difference between "clinical" significance and "statistical" significance. But that's problem for another day.

In [0]:
###C.P. v2 Example

###################################################Acquire data (1)###################################################
!pip install -q xlrd
%cd bwsi_testsets
!git clone https://github.com/lylemp/bwsi_testsets.git #with public datasets just replace URL
!ls bwsi_testsets #shows the files in this folder. pick the dataset you want
import pandas as po
dataset=po.read_csv('bwsi_testsets/diabetes.csv')


###################################################cleaning data, data preanalysis (2)###################################################
dataset.isnull().sum()
dataset['Outcome']=dataset['Outcome'].astype('int64')
dataset[dataset.columns!="?"]

dataset.info()

dataset.describe()

###################################################data visualization (3)###################################################
import seaborn as sns
import matplotlib.pyplot as py
dataset.corr()
sns.heatmap(dataset.corr(),annot=True,cbar=True) #if seaborn is allowed
py.matshow(dataset.corr())
sns.pairplot(dataset)
#etc.

###################################################model selection (4)###################################################
from sklearn.model_selection import train_test_split
outcome=dataset['Outcome']
data=dataset[dataset.columns[:8]]
train,test=train_test_split(dataset,test_size=0.25,random_state=0,stratify=dataset['Outcome'])# outcome stratification (>explain why)

train_X=train[train.columns[:8]]
test_X=test[test.columns[:8]]
train_Y=train['Outcome']
test_Y=test['Outcome']


from sklearn import metrics


from sklearn import svm
types=['rbf','linear']
for i in types:
    svm_model=svm.SVC(kernel=i)
    svm_model.fit(train_X,train_Y)
    svm_prediction=svm_model.predict(test_X)
    svm_acc=metrics.accuracy_score(svm_prediction,test_Y)
    print('Accuracy for SVM kernel=',i,'is',svm_acc)


from sklearn.neighbors import KNeighborsClassifier
a_index=list(range(1,11))
a=pd.Series()
x=[0,1,2,3,4,5,6,7,8,9,10]
for i in list(range(1,11)):
    kn_model=KNeighborsClassifier(n_neighbors=i) 
    kn_model.fit(train_X,train_Y)
    kn_prediction=kn_model.predict(test_X)
    a=a.append(pd.Series(metrics.accuracy_score(kn_prediction,test_Y)))
plt.plot(a_index, a)
plt.xticks(x)
plt.show()
print('Accuracies for different values of n are:',a.values)

from sklearn.tree import DecisionTreeClassifier
dt_model=DecisionTreeClassifier()
dt_model.fit(train_X,train_Y)
lr_prediction=dt_model.predict(test_X)
dt_acc=metrics.accuracy_score(lr_prediction,test_Y)
print('The accuracy of the Decision Tree is',dt_acc)


from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
lr_model.fit(train_X,train_Y)
lr_prediction=lr_model.predict(test_X)
lr_acc=metrics.accuracy_score(lr_prediction,test_Y)
print('The accuracy of the Logistic Regression is',lr_acc)

#maybe introduce K-fold cross-validation?


###################################################model evaluation (5)###################################################
from scipy.stats import norm
def twopropztest(prop1,n1,prop2,n2,sign):
  if(sign=='>'):
    z=(prop1-prop2-0)/((prop1*(1-prop1)/n1)+(prop2*(1-prop2)/n2))**(1/2) 
    print(z)
    return 1-norm.cdf(z,0,1)
  
!pip install -q xlrd
!git clone https://github.com/lylemp/bwsi_testsets.git #with public datasets just replace URL
!ls bwsi_testsets #shows the files in this folder. pick the dataset you want
import pandas as po
dataset=po.read_csv('bwsi_testsets/diabetes.csv')

twopropztest(lr_acc,len(dataset),dt_acc,len(dataset),'>')


expected=len(dataset[dataset.Outcome==1])/len(dataset)

###################################################Conclusion (6)###################################################
#there is significant evidence to suggest that the logistic regression model performs the best with an accuracy of 76% at the 5% alpha level

Things to consider for your conclusion:

* Compare the hypothesis to the results; do the data support or reject the hypothesis? 
* What could you revise in the methods to improve the experiment? 
* What might cause the relationship between the dependent variable and the independent variables (confounding variables, outliers)? What other variables can you think of measuring to improve your model?
* What questions were raised; are there more scientific tests that could be performed?
* If any statistical tests were performed, be sure to include a discussion about them.

In [78]:
from sklearn.model_selection import train_test_split
outcome=dataset['Outcome']
data=dataset[dataset.columns[:8]]
train,test=train_test_split(dataset,test_size=0.25,random_state=0,stratify=dataset['Outcome'])# outcome stratification (>explain why)

train_X=train[train.columns[:8]]
test_X=test[test.columns[:8]]
train_Y=train['Outcome']
test_Y=test['Outcome']


from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier

dt_model=DecisionTreeClassifier()
dt_model.fit(train_X,train_Y)
dt_prediction=dt_model.predict(test_X)
dt_acc=metrics.accuracy_score(dt_prediction,test_Y)
print('The accuracy of the Decision Tree is',dt_acc)


from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
lr_model.fit(train_X,train_Y)
dt_prediction=lr_model.predict(test_X)
lr_acc=metrics.accuracy_score(dt_prediction,test_Y)
print('The accuracy of the Logistic Regression is',lr_acc)

#maybe introduce K-fold cross-validation? (might as well if going this in-depth)

from scipy.stats import norm
def twopropztest(prop1,n1,prop2,n2,sign):
  if(sign=='>'):
    z=(prop1-prop2-0)/((prop1*(1-prop1)/n1)+(prop2*(1-prop2)/n2))**(1/2) 
    print('%.3f'%(z))
    return ('%.3f' % (1-norm.cdf(z,0,1)))
  
twopropztest(lr_acc,len(dataset),dt_acc,len(dataset),'>')

The accuracy of the Decision Tree is 0.7604166666666666
The accuracy of the Logistic Regression is 0.7760416666666666
0.726


'0.234'

In [81]:
expected=len(dataset[dataset.Outcome==1])/len(dataset)
expected2=len(dataset[dataset.Outcome==0])/len(dataset)
expected
expected2

0.6510416666666666

Challenge Project v.3

---

Have student predict a certain outcome by creating surveys to generate data and building a model/selecting a model that gives the highest accuracy.

"I liken predicting to fortune-telling. How amazing it would be to have a machine that can tell you the results of events that haven't happened yet?"


In [72]:

#cleaning data, data preanalysis (2)

dataset.isnull().sum()
dataset['Outcome']=dataset['Outcome'].astype('int64')
for i in range(1,len(dataset)):
  for j in range(1,len(dataset.columns)):
    if(dataset.iloc[i,j]==0):
      dataset.drop(dataset.index[i])
      continue
dataset      

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1
